In [8]:
"""
https://stackoverflow.com/questions/39955521/sqlalchemy-existing-database-query
https://stackoverflow.com/questions/4617291/how-do-i-get-a-raw-compiled-sql-query-from-a-sqlalchemy-expression
""";

In [9]:
from sqlalchemy import func
from sqlalchemy import or_
from sqlalchemy import and_

from sqlalchemy import create_engine
from sqlalchemy import Column, Date, Integer, String, ForeignKey

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session, scoped_session, sessionmaker

In [10]:
Base = declarative_base()
DBSession = scoped_session(sessionmaker())

In [11]:
engine = create_engine("sqlite:///inventory.db", echo=True)

In [12]:
Base = automap_base()
Base.prepare(engine, reflect=True);

Inventory = Base.classes.inventory
session = Session(engine)

2021-07-29 23:00:25,765 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-07-29 23:00:25,766 INFO sqlalchemy.engine.base.Engine ()
2021-07-29 23:00:25,768 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-07-29 23:00:25,769 INFO sqlalchemy.engine.base.Engine ()
2021-07-29 23:00:25,770 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-07-29 23:00:25,771 INFO sqlalchemy.engine.base.Engine ()
2021-07-29 23:00:25,774 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("inventory")
2021-07-29 23:00:25,776 INFO sqlalchemy.engine.base.Engine ()
2021-07-29 23:00:25,777 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'inventory' AND type = 'table'
2021-07-29 23:00:25,781 INFO sqlalchemy.engine.base.Engine ()
2021-07-29 23:00:25,783 INFO sqlalchemy

In [13]:
q = (DBSession
     .query(Inventory.id, Inventory.sku, func.count(Inventory.pprice).label("value0"))
     .filter(and_(Inventory.id.in_((1, 2, 3, 4, 5)), ~Inventory.id.in_((6, 7, 8, 9)))) 
     .group_by(Inventory.id, Inventory.sku)
     .order_by(Inventory.id, Inventory.sku)
    )

In [14]:
str(q.statement.compile(compile_kwargs={"literal_binds": True}))

'SELECT inventory.id, inventory.sku, count(inventory.pprice) AS value0 \nFROM inventory \nWHERE inventory.id IN (1, 2, 3, 4, 5) AND inventory.id NOT IN (6, 7, 8, 9) GROUP BY inventory.id, inventory.sku ORDER BY inventory.id, inventory.sku'